## Project Template (Deepnote)

You can use this notebook as a starting point for your class project and/or the extra exercise for graduate students in Deepnote. It comes set up with the dependencies we use in class, so you don't have to install these yourself.

**Notes:**
- To use this in your own workspace, click “Duplicate” for the project in Deepnote.
- Class dependencies, namely `drake` and the `manipulation` package, are already preinstalled in the Deepnote image for this course, as well as a bunch of other dependencies that these packages depend on or that we use in class
- To see the full list of installed dependencies, as well as version numbers etc, see this file: [pyproject.toml](https://github.com/RussTedrake/manipulation/blob/master/pyproject.toml).
  - **Note**: we install all the extra dependencies, including the `dev` dependencies, into the Docker image that this deepnote project runs on.


In [2]:
%load_ext autoreload
%autoreload 2
# ^ make it so we can change inner files


In [3]:

import numpy as np
from pydrake.all import (
    ContactModel,
    DiagramBuilder,
    MultibodyPlant,
    RigidTransform,
    RollPitchYaw,
    SceneGraph,
    Simulator,
    StartMeshcat,
    VectorLogSink,
)

import manipulation
from manipulation.station import (
    AddPointClouds,
    MakeHardwareStation,
    Parser,
)
from scripts.load_scenario import load_scenario

In [4]:
# Start meshcat for visualization
meshcat = StartMeshcat()
print("Click the link above to open Meshcat in your browser!")

INFO:drake:Meshcat listening for connections at http://localhost:7001


Click the link above to open Meshcat in your browser!


In [5]:
use_hydroelastic = False # this takes forever

scenario = load_scenario()

# since makehardwarestation finalizes the plant before giving it back to us
def callback(parser: Parser):
    plant = parser.plant()
    plant.set_contact_model(ContactModel.kHydroelasticWithFallback)
    scene_graph: SceneGraph = parser.scene_graph()
    config = scene_graph.get_config()
    if use_hydroelastic:
        config.default_proximity_properties.compliance_type = "compliant"
    scene_graph.set_config(config)

station = MakeHardwareStation(scenario, meshcat=meshcat, parser_prefinalize_callback=callback)
plant: MultibodyPlant = station.plant()


builder = DiagramBuilder()
builder.AddSystem(station)

to_point_cloud = AddPointClouds(
    scenario=scenario, station=station, builder=builder, meshcat=meshcat
)
builder.ExportOutput(
    to_point_cloud["camera0"].point_cloud_output_port(), "point_cloud0"
)

diagram = builder.Build()




In [ ]:

simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(2)

In [ ]:
from scripts.grasp_broom import plan_path

# basic test
X_WG1 = RigidTransform(RollPitchYaw(np.pi/2, 0, 0), [0, 0.5, 1])
X_WG2 = RigidTransform(RollPitchYaw(np.pi/2, 0, np.pi/2), [0, 1, 1])

